In [1]:
import csv
import os, random
import glob
from pydub import AudioSegment

os.chdir(path=r"D:\03_batch_3")
print("Current Working Directory " , os.getcwd())

Current Working Directory  D:\03_batch_3


C:\Users\koool\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Step 1: Open dir and Select random text file
def select_random_file():
    os.chdir(dataset_path)
    myFiles = glob.glob('*.txt')
    text_file = random.choice(myFiles) #change dir name to whatever
    audio_file = text_file.replace(".txt", ".wav")
    read_random_line(text_file, audio_file)

# Step 2: Read random line
def read_random_line(text_file, audio_file):
    global total_duration
    global current_duration    

    try:
        lines = open(dataset_path + text_file, encoding="utf8").read().splitlines()
        random_line = random.choice(lines)

        columns = random_line.split("\t")
        start = float(columns[2])
        end = float(columns[3])
        transcript = columns[4]
        duration = end - start

        segment_name = audio_file.replace(".wav", "") + '_' + str(int(start))  + '_' + str(int(end)) + ".wav"
     #   print(columns)

     #   print(segment_name, duration)
        # if segment is not valid read another one
        if segment_name in files and duration < 1:
            read_random_line(text_file, audio_file)
        else:
            total_duration += duration 
            current_duration += duration
            split_audio(audio_file, start, end,transcript, segment_name)
    except ValueError:
        read_random_line(text_file, audio_file)


# Step 3: Split the audio into segment 
def split_audio(audio_file, start, end, transcript, segment_name):
    t1 = start * 1000 #Works in milliseconds
    t2 = end * 1000
    new_audio_path = current_path+segment_name
    new_audio = AudioSegment.from_wav(dataset_path + audio_file)
    new_audio = new_audio[t1:t2]
    new_audio.export(new_audio_path, format="wav") #Exports to a wav file in the current path.
    write_segment(new_audio_path, transcript)
    write_segment_excel(segment_name,current_path,audio_file,start,end,transcript)

# Step 4: Write the new segment and write in the CSV
def write_segment(segment_name, transcript):
    writer.writerow([segment_name[2:], transcript])
    files.append(segment_name)

# Step 5: Write the new segment and write in the excel
def write_segment_excel(segment_name,current_path,audio_file,start,end,transcript):
    r_start = int(start)
    r_end = int(end)
    segment_name = audio_file.replace(".wav", "") + '_' + str(r_start)  + '_' + str(r_end) + ".wav"
    audio_file = dataset_path + audio_file
    current_path = current_path + segment_name
    writer_excel.writerow([segment_name,current_path[2:],audio_file[2:],start,end,transcript])
    #print(segment_name," ",current_path," ",audio_file," ",start," ",end," ",transcript)

In [3]:
for f in sorted(os.listdir(os.getcwd())):
     
    ######################[CAN BE UPDATED]###################
    dialect = f
    # The root folder where the dialect is exist
    dataset_path = r"D:/03_batch_3/{}/".format(dialect)
    wanted_duration = 0.84 #in hours
    number_of_dir = 10 #total number of files you want to separate the segments to 
    ########################################################

    output_path = r'D:/batch_sample_3/{}_segments/'.format(dialect)

    ######################[DO NOT CHANGE]###################
    files = []
    segments_path = output_path
    dir_counter = 0
    for root, dirs, files in os.walk(segments_path, topdown=False):
        dir_counter = len(dirs) + 1
        break

    current_path = segments_path + str(dir_counter) + "/"
    total_duration = 0
    current_duration = 0

    if not os.path.exists(segments_path):
        os.mkdir(segments_path)

    if not os.path.exists(current_path):
        os.mkdir(current_path)

    # Create CSV
    header = ['segment path', 'transcript']
    f = open(segments_path + str(dialect) + "_segments.csv", "w", encoding="utf8")
    writer = csv.writer(f)
    writer.writerow(header)

    # Create excel
    header_excel = ['segment_name','current_path','audio_file','start','end','transcript']
    f_excel = open(segments_path + str(dialect) + "_details.csv", "w", encoding="utf8")
    writer_excel = csv.writer(f_excel)
    writer_excel.writerow(header_excel)

    ########################################################


    if __name__ == "__main__":
        while total_duration < wanted_duration * 60 * 60:
            if current_duration > wanted_duration* 60 * 60 / number_of_dir:
                dir_counter += 1
                current_path = segments_path + str(dir_counter) + "/"
                current_duration = 0

                if not os.path.exists(current_path):
                    os.mkdir(current_path)
            
            print(total_duration)
            select_random_file()



0
7.97495448059
31.024579927450006
36.524433893600005
49.573646493423
52.073086035516994
58.322483625878995
87.09289187055899
96.26654107812898
107.36578597608799
122.18984747280999
129.26366025231098
132.113193345881
158.6366586852758
183.11066733949576
184.21063813272505
187.68554586588104
207.88426709582106
213.20879876252104
237.27763662345106
243.20176431335108
244.77666928229107
246.1515900226111
270.2204278835411
278.2449198573146
301.61844009245056
302.41839182271053
306.84307184174054
319.39227735835806
333.14106804701805
338.3656085087081
364.81173364223105
390.20523236154304
391.055149654783
402.854763575073
413.453554909173
436.597629529333
444.697078508925
448.62191952906
456.59513166834
470.54390604894
473.96840180217004
475.04316744198
492.816140646088
495.01591128031805
500.39035401480805
512.814604325368
529.53865790627
557.41076790141
561.4853032491801
588.6579303184911
618.4569034404722
630.2308752432722
643.1766401415282
661.9008639138582
666.9258342108682
677.90077

KeyboardInterrupt: 